<a href="https://colab.research.google.com/github/baskayj/Boundary-behavior-in-stochastic-differential-equations-used-in-Finance/blob/master/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

from collections import namedtuple
from google.colab import drive

mc_rep = 10000                                                                  #Number of generated trajectories

In [0]:
#To load and save simulation data

drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/My Drive/MSc. Szakdolgozat/Data/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#CIR MODEL
#There are multiple finite-diffrence schemes for simulation including,
#Euler-Maryama Scheme: X[i] = X[i-1] + kappa*(theta-X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt)
#Weighted Milstein Scheme: X[i] = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
#To be continued...

#gamma = 2*kappa*theta/sigma^2

#Path(returns the entire path as a NumPy array)
def CIR_path(X0, dt, T, kappa, theta, sigma, seed):
  N = int(T/dt)
  alpha = 0.8
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    x = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
    if x < 0 :
      X[i] = 0
    else:
      X[i] = x
  return X

#Process(returns only the values at t = T)
def CIR_prcss(X0, dt, T, kappa, theta, sigma, seed):
  N = int(T/dt)
  alpha = 0.8
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    x = (X[i-1] + kappa*(theta-alpha*X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) + (1/4)*np.power(sigma,2)*dt*(np.power(W[i-1],2)-1))/(1+(1-alpha)*kappa*dt)
    if x < 0 :
      X[i] = 0
    else:
      X[i] = x
  return X[N]

In [0]:
# CLASSES for MP

#CIR
class CIR():
  def __init__(self,X0,dt,T,kappa,theta,sigma):
    self.X0 = X0
    self.dt = dt
    self.T = T
    self.kappa = kappa
    self.theta = theta
    self.sigma = sigma
  def Path(self,seed):
    return CIR_path(self.X0, self.dt, self.T, self.kappa, self.theta, self.sigma, seed)
  def Prcss(self,seed):
    return CIR_prcss(self.X0, self.dt, self.T, self.kappa, self.theta, self.sigma, seed)

In [0]:
#MULTIPROCESS FUNCTIONS

def CIR_Path_mp(X0,dt,T,kappa,theta,sigma):
  cir = CIR(X0,dt,T,kappa,theta,sigma)
  pool = mp.Pool(processes=mp.cpu_count())
  return pool.map(cir.Path,range(mc_rep))

def CIR_Prcss_mp(X0,dt,T,kappa,theta,sigma):
  cir = CIR(X0,dt,T,kappa,theta,sigma)
  pool = mp.Pool(processes=mp.cpu_count())
  return pool.map(cir.Prcss,range(mc_rep))

------

# Relationship between the CIR-Process and the Noncentral $\chi ^2$-distribution

References:

---

# EXPERIMENT: T -> infty

In [0]:
#EXPERIMENT:

model = 'CIR_'

name = 'test'

#Parameters:
all_params = namedtuple('all_params','X0 dt T kappa theta sigma')
P = all_params(X0 = 0.5, dt = 1/10000, T = 1,kappa = 1, theta = 1, sigma = 0.6)

#Variable:
var_name = 'gamma'
var = [2]
kappa = list(map(lambda p: p*np.power(P.sigma,2)*(1/(2*P.theta)),Var))


start_time = time.time()
#GENERATING ALL TRAJECTORIES
X = np.zeros((len(Var),mc_rep))
for i in range(len(kappa)):
  print(f"\n Staring simulations on {Var_name} = {Var[i]}\n")
  s_time = time.time()
  j = 0
  for x in CIR_Prcss_mp(P.X0,P.dt,P.T,kappa[i],P.theta,P.sigma):
    X[i,j] = x
    j += 1
  e_time = time.time()
  print(f"Finished generating trajectories for {Var_name} = {var[i]} in {e_time-s_time} seconds.\n")


#Saving the Data
np.save(f"{DATA_PATH}{model}{name}_params.npy",P) #!!
np.save(f"{DATA_PATH}{model}{name}_{var_name}_values.npy",var) #!!
np.save(f"{DATA_PATH}{model}{name}.npy",X)
print(f"Data saved to {DATA_PATH} succefully. \n")

#Memory Management
del X

end_time = time.time()
print(f"The simulation finished in {end_time-start_time} seconds.\n")

# EXPERIMENT: X0 -> 0

# EXPERIMENT: Using the Noncentral $\chi ^2$-distribution

#EXPERIMENT: Finding the critical $\gamma$-values numerically

# OLD CODE

In [0]:
#OUTDATED!!!!!!!!!
def CIR_path_Euler(X0,N,T,repeat, kappa,theta,sigma,seed, boundary):
  dt = (T/N)*repeat
  np.random.seed(seed)
  W = np.random.normal(size=N)
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    if i%repeat == 0:
      X[i] = boundary(X[i-1], X[i-1] + kappa*(theta-X[i-1])*dt + sigma*np.sqrt(X[i-1])*W[i-1]*np.sqrt(dt) )
    else:
      X[i] = X[i-1]
  return X

#CEV
#OUTDATED!!!!!!!!!
def CEV_path(X0,N,T,repeat, W, boundary, mu,sigma,beta):                              # beta in (-1,0)
  dt = (T/N)*repeat
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    if i%repeat == 0:
      X[i] = boundary(X[i-1], X[i-1] + mu*X[i-1]*dt + sigma*np.pow(X[i-1],beta+1)*W[i-1]*np.sqrt(dt) )
    else:
      X[i] = X[i-1]
  return X


#GBM
#OUTDATED!!!!!!!!
def GBM_path(X0,N,T,repeat, W, boundary, mu,sigma):
  dt = (T/N)*repeat
  X = np.zeros(N)
  X[0] = X0
  for i in range(1,N):
    if i%repeat == 0:
      X[i] = boundary(X[i-1], X[i-1] + mu*X[i-1]*dt + sigma*X[i-1]*W[i-1]*np.sqrt(dt) )
    else:
      X[i] = X[i-1]
  return X


#BOUNDARY CONDITIONS

def no_boundary(Xi,Xii):
  return Xii


def sticky_boundary(Xi,Xii):
  if Xi < B:
    return Xi
  elif Xii < B:
    return 0 #!!!!!!!!!!!!!!!!!!!!!
  else:
    return Xii


def reflective_boundary(Xi,Xii):
  if Xii <= B:
    return (2*B-Xii)
  elif Xii >= A:
    return (2*A-Xii)
  else:
    return Xii

## From bulk to boundary:

### 10k

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#Part I.
#From bulk to boundary
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_bulk_10k'

  part = '_part_1'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 10000, T = 1,kappa = 1, theta = 1, sigma = 0.6, alpha = 0.8)

  gamma = [-2.5,-2,-1.5,-1,-0.5] #When doing 10k only 6 can be done in a single measurment! Please divide them into parts.
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + part + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + part + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + part + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = -2.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -2.5 in 450.2725296020508 seconds.


 Staring simulations on gamma = -2



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -2 in 550.5965433120728 seconds.


 Staring simulations on gamma = -1.5



...

Finished generating trajectories for gamma = -1.5 in 567.3017845153809 seconds.


 Staring simulations on gamma = -1



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -1 in 547.9143452644348 seconds.


 Staring simulations on gamma = -0.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -0.5 in 575.2237040996552 seconds.

Data saved to /content/gdrive/My Drive/MSc. Szakdolgozat/Data/ succefully. 

The simulation finished in 2785.619341135025 seconds.



In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#Part I.
#From bulk to boundary
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_bulk_10k'

  part = '_part_2'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 10000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [-0.2,-0.1,0,0.1,0.2] #When doing 10k only 6 can be done in a single measurment! Please divide them into parts.
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + part + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + part + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + part + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = -0.2



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -0.2 in 462.17609548568726 seconds.


 Staring simulations on gamma = -0.1



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -0.1 in 555.7196614742279 seconds.


 Staring simulations on gamma = 0



...

Finished generating trajectories for gamma = 0 in 559.1353735923767 seconds.


 Staring simulations on gamma = 0.1



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0.1 in 544.9100868701935 seconds.


 Staring simulations on gamma = 0.2



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0.2 in 538.1378140449524 seconds.

Data saved to /content/gdrive/My Drive/MSc. Szakdolgozat/Data/ succefully. 

The simulation finished in 2758.432195186615 seconds.



In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#Part I.
#From bulk to boundary
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_bulk_10k'

  part = '_part_3'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 10000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [0.5,0.8,0.9,1,1.1] #When doing 10k only 6 can be done in a single measurment! Please divide them into parts.
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + part + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + part + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + part + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = 0.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0.5 in 541.6124782562256 seconds.


 Staring simulations on gamma = 0.8

Finished generating trajectories for gamma = 0.8 in 544.0265665054321 seconds.


 Staring simulations on gamma = 0.9

Finished generating trajectories for gamma = 0.9 in 541.2234892845154 seconds.


 Staring simulations on gamma = 1

Finished generating trajectories for gamma = 1 in 552.5006415843964 seconds.


 Staring simulations on gamma = 1.1

Finished generating trajectories for gamma = 1.1 in 559.4469728469849 seconds.

Data saved to /content/gdrive/My Drive/MSc. Szakdolgozat/Data/ succefully. 

The simulation finished in 2841.5799827575684 seconds.



In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#Part I.
#From bulk to boundary
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_bulk_10k'

  part = '_part_4'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 10000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [1.2,1.5,2,2.5,3] #When doing 10k only 6 can be done in a single measurment! Please divide them into parts.
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + part + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + part + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + part + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = 1.2

Finished generating trajectories for gamma = 1.2 in 444.7773389816284 seconds.


 Staring simulations on gamma = 1.5

Finished generating trajectories for gamma = 1.5 in 534.6602892875671 seconds.


 Staring simulations on gamma = 2

Finished generating trajectories for gamma = 2 in 550.8035087585449 seconds.


 Staring simulations on gamma = 2.5

Finished generating trajectories for gamma = 2.5 in 550.3296642303467 seconds.


 Staring simulations on gamma = 3

Finished generating trajectories for gamma = 3 in 542.2073836326599 seconds.

Data saved to /content/gdrive/My Drive/MSc. Szakdolgozat/Data/ succefully. 

The simulation finished in 2720.871908187866 seconds.



### 1k

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From bulk to boundary
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  start_time = time.time()
  model = 'CIR_'

  name = 'phase_diagram_from_bulk_1k'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 1000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [-2.5,-2,-1.5,-1,-0.5,-0.2,-0.1,0,0.1,0.2,0.5,0.8,0.9,1,1.1,1.2,1.5,2,2.5,3]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = -2.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -2.5 in 53.25004529953003 seconds.


 Staring simulations on gamma = -2



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -2 in 53.41324257850647 seconds.


 Staring simulations on gamma = -1.5



...

Finished generating trajectories for gamma = 0.1 in 54.1777880191803 seconds.


 Staring simulations on gamma = 0.2



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0.2 in 53.22897219657898 seconds.


 Staring simulations on gamma = 0.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0.5 in 53.11545276641846 seconds.


 Staring simulations on gamma = 0.8

Finished generating trajectories for gamma = 0.8 in 53.18507719039917 seconds.


 Staring simulations on gamma = 0.9

Finished generating trajectories for gamma = 0.9 in 53.60061049461365 seconds.


 Staring simulations on gamma = 1

Finished generating trajectories for gamma = 1 in 53.22264885902405 seconds.


 Staring simulations on gamma = 1.1

Finished generating trajectories for gamma = 1.1 in 53.11958694458008 seconds.


 Staring simulations on gamma = 1.2

Finished generating trajectories for gamma = 1.2 in 53.33901071548462 seconds.


 Staring simulations on gamma = 1.5

Finished generating trajectories for gamma = 1.5 in 53.44973421096802 seconds.


 Staring simulations on gamma = 2

Finished generating trajectories for gamma = 2 in 53.07749962806702 seconds.


 Staring simulations on gamma = 2.5

Finished generating trajectories for gamma = 2.5 in 53.449497461

### 100

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From bulk to boundary
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_bulk_100'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 100, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [-2.5,-2,-1.5,-1,-0.5,-0.2,-0.1,0,0.1,0.2,0.5,0.8,0.9,1,1.1,1.2,1.5,2,2.5,3]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = -2.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -2.5 in 5.358294486999512 seconds.


 Staring simulations on gamma = -2



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -2 in 5.29456353187561 seconds.


 Staring simulations on gamma = -1.5



...

Finished generating trajectories for gamma = 0.1 in 5.287887334823608 seconds.


 Staring simulations on gamma = 0.2



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0.2 in 5.3165507316589355 seconds.


 Staring simulations on gamma = 0.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0.5 in 5.354166507720947 seconds.


 Staring simulations on gamma = 0.8

Finished generating trajectories for gamma = 0.8 in 5.340635776519775 seconds.


 Staring simulations on gamma = 0.9

Finished generating trajectories for gamma = 0.9 in 5.293152809143066 seconds.


 Staring simulations on gamma = 1

Finished generating trajectories for gamma = 1 in 5.313228368759155 seconds.


 Staring simulations on gamma = 1.1

Finished generating trajectories for gamma = 1.1 in 5.32389235496521 seconds.


 Staring simulations on gamma = 1.2

Finished generating trajectories for gamma = 1.2 in 5.421584844589233 seconds.


 Staring simulations on gamma = 1.5

Finished generating trajectories for gamma = 1.5 in 5.364306211471558 seconds.


 Staring simulations on gamma = 2

Finished generating trajectories for gamma = 2 in 5.312141418457031 seconds.


 Staring simulations on gamma = 2.5

Finished generating trajectories for gamma = 2.5 in 5.32565498352

### 10

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From bulk to boundary
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_bulk_10'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 10, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [-2.5,-2,-1.5,-1,-0.5,-0.2,-0.1,0,0.1,0.2,0.5,0.8,0.9,1,1.1,1.2,1.5,2,2.5,3]

  #Generating ALL trajectories
  X = []
  for j in range(0,len(gamma)):
    print(f"\n Staring simulations on gamma = {gamma[j]}\n")
    s_time = time.time()
    kappa = gamma[j]*np.power(pp.sigma,2)*(1/(2*pp.theta))
    X.append(CIR_mp(pp.X0,pp.n_grid,pp.T,kappa,pp.theta,pp.sigma,pp.alpha,sticky_boundary))
    e_time = time.time()-s_time
    print(f"Finished generating trajectories for gamma = {gamma[j]} in {e_time} seconds.\n")
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X

----

##From boundary to bulk

### 10k

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From boundary to bulk
#Part I.
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_boundary_10k'

  part = '_part_1'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0, n_grid = 10000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [-2,-1,0,0.5,0.6]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + part + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + part + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + part + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = -2



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -2 in 564.5745844841003 seconds.


 Staring simulations on gamma = -1



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = -1 in 669.0690138339996 seconds.


 Staring simulations on gamma = 0



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0 in 659.4414763450623 seconds.


 Staring simulations on gamma = 0.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in sqrt


Finished generating trajectories for gamma = 0.5 in 600.6867589950562 seconds.


 Staring simulations on gamma = 0.6

Finished generating trajectories for gamma = 0.6 in 553.723103761673 seconds.

Data saved to /content/gdrive/My Drive/MSc. Szakdolgozat/Data/ succefully. 

The simulation finished in 3148.1829755306244 seconds.



In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From boundary to bulk
#Part II.
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_boundary_10k'

  part = '_part_2'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0, n_grid = 10000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [0.7,0.8,0.9,1,1.1]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + part + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + part + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + part + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = 0.7

Finished generating trajectories for gamma = 0.7 in 559.3520951271057 seconds.


 Staring simulations on gamma = 0.8

Finished generating trajectories for gamma = 0.8 in 550.8121597766876 seconds.


 Staring simulations on gamma = 0.9

Finished generating trajectories for gamma = 0.9 in 551.5026867389679 seconds.


 Staring simulations on gamma = 1



In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From boundary to bulk
#Part III.
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_boundary_10k'

  part = '_part_3'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0, n_grid = 10000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [1.2,1.3,1.4,1.5,2]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + part + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + part + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + part + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From boundary to bulk
#Part IV.
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_boundary_10k'
 
  part = '_part_4'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0, n_grid = 10000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [2.5,3,3.5,4,5] 
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + part + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + part + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + part + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = 2.5

Finished generating trajectories for gamma = 2.5 in 0.6426856517791748 seconds.


 Staring simulations on gamma = 3

Finished generating trajectories for gamma = 3 in 0.629406213760376 seconds.


 Staring simulations on gamma = 3.5

Finished generating trajectories for gamma = 3.5 in 0.6331160068511963 seconds.


 Staring simulations on gamma = 4

Finished generating trajectories for gamma = 4 in 0.6747739315032959 seconds.


 Staring simulations on gamma = 5

Finished generating trajectories for gamma = 5 in 0.6549215316772461 seconds.

Data saved to /content/gdrive/My Drive/MSc. Szakdolgozat/Data/ succefully. 

The simulation finished in 3.2695608139038086 seconds.



### 1k

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From boundary to bulk
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_boundary_1k'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0, n_grid = 1000, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [-2,-1,0,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,2,2.5,3,3.5,4,5]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X

### 100

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From boundary to bulk
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_boundary_100'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0, n_grid = 100, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [-2,-1,0,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,2,2.5,3,3.5,4,5]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X

### 10

In [0]:
#EXPERIMENT: Critical Behaviour (Feller Condition)
#From boundary to bulk
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()

  model = 'CIR_'

  name = 'phase_diagram_from_boundary_10'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T kappa theta sigma alpha')
  pp = all_params(X0 = 0, n_grid = 10, T = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [-2,-1,0,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,2,2.5,3,3.5,4,5]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X

---

## Alpha

Q:How does the alpha parameter influence the distribution?

A:It doesn't.

In [0]:
#EXPERIMENT: Alpha dependance of the model
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()
  model = 'CIR_'

  name = 'alpha_dependance'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T repeat kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 1000, T = 1, repeat = 1,kappa = 2.4, theta = 0.3, sigma = 0.6, alpha = 0.8)

  alpha = [0,0.1,0.5,0.8,1]

  #Generating ALL trajectories
  X = np.zeros((len(alpha),mc_rep,pp.n_grid))
  for i in range(len(alpha)):
    print(f"\n Staring simulations on alpha = {alpha[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_mp(pp.X0,pp.n_grid,pp.T,pp.kappa,pp.theta,pp.sigma,alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for alpha = {alpha[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',alpha) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X

---

# MISC. TEST

In [0]:
#EXPERIMENT: Testing modified code
#gamma = 2*kappa*theta/sigma^2

if __name__ == '__main__':
  start_time = time.time()
  model = 'CIR_'

  name = 'test'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T repeat kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 10000, T = 1, repeat = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [2]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = np.zeros((len(gamma),mc_rep,pp.n_grid))
  for i in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[i]}\n")
    s_time = time.time()
    j = 0
    for x in CIR_Pool(pp.X0,pp.n_grid,pp.T,pp.repeat,kappa[i],pp.theta,pp.sigma,pp.alpha,sticky_boundary):
      X[i,j,:] = x
      j += 1
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[i]} in {e_time-s_time} seconds.\n")




  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',gamma) # SAVING THE USED PARAMETERS!!!
  np.save(DATA_PATH + 'CIR_' + name + '.npy',X)
  print(f"Data saved to {DATA_PATH} succefully. \n")

  end_time = time.time()
  print(f"The simulation finished in {end_time-start_time} seconds.\n")
  del X


 Staring simulations on gamma = 2

Finished generating trajectories for gamma = 2 in 553.0340313911438 seconds.

Data saved to /content/gdrive/My Drive/MSc. Szakdolgozat/Data/ succefully. 

The simulation finished in 556.5355095863342 seconds.



In [0]:
if __name__ == '__main__':
  start_time = time.time()
  model = 'CIR_'

  name = 'alpha_dependance'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T repeat kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 10000, T = 1, repeat = 1,kappa = 2.4, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [2]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = []
  for j in range(0,len(gamma)):
    print(f"\n Staring simulations on gamma = {gamma[j]}\n")
    s_time = time.time()
    X.append(CIR_mp(pp.X0,pp.n_grid,pp.T,pp.repeat,kappa[j],pp.theta,pp.sigma,pp.alpha,sticky_boundary))
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[j]} in {e_time-s_time} seconds.\n")

  #Converting the list into a NumPy array, for easier usage
  S = np.zeros((len(gamma),mc_rep,pp.n_grid))
  j = 0
  for x in X:
    k = 0
    for i in x:
      S[j,k,:] = i
      k += 1
    j += 1
  end_time = time.time()
  del X #DONT FORGET TO REMOVE THIS LINE!!!!
  del S #DONT FORGET TO REMOVE THIS LINE!!!!
  print(f"Total time:{end_time-start_time}")


 Staring simulations on gamma = 2

Finished generating trajectories for gamma = 2 in 556.0748794078827 seconds.

Total time:556.7253556251526


In [0]:
if __name__ == '__main__':
  start_time = time.time()
  model = 'CIR_'

  name = 'test'

  #Iterable parameters:
  all_params = namedtuple('all_params','X0 n_grid T repeat kappa theta sigma alpha')
  pp = all_params(X0 = 0.5, n_grid = 10000, T = 1, repeat = 1,kappa = 0.3, theta = 0.3, sigma = 0.6, alpha = 0.8)

  gamma = [2]
  kappa = list(map(lambda p: p*np.power(pp.sigma,2)*(1/(2*pp.theta)),gamma))

  #Generating ALL trajectories
  X = []
  for j in range(len(kappa)):
    print(f"\n Staring simulations on gamma = {gamma[j]}\n")
    s_time = time.time()
    x = []
    for i in range(mc_rep):
      x.append(CIR_path(pp.X0, pp.n_grid, pp.T, pp.repeat, kappa[j], pp.theta, pp.sigma, pp.alpha, i, sticky_boundary))
    X.append(x)
    e_time = time.time()
    print(f"Finished generating trajectories for gamma = {gamma[j]} in {e_time-s_time} seconds.\n")
  S = np.zeros((len(gamma),mc_rep,pp.n_grid))
  j = 0
  for x in X:
    k = 0
    for i in x:
      S[j,k,:] = i
      k += 1
    j += 1
  end_time = time.time()
  del X #DONT FORGET TO REMOVE THIS LINE!!!!
  del S #DONT FORGET TO REMOVE THIS LINE!!!!
  print(f"Total time:{end_time-start_time}")


 Staring simulations on gamma = 2

Finished generating trajectories for gamma = 2 in 1183.7417204380035 seconds.

Total time:1184.133409023285


In [0]:
for j in range(0,np.shape(S)[0]):
  plt.figure()
  plt.title("Gamma = " + str(gamma[j]))
  n,bins,patches = plt.hist(S[j,:,(np.shape(S)[2]-1)],bins = 200, density = True, histtype = "step")
  plt.show()

[None]

In [0]:

  #Converting the list into a NumPy array, for easier usage
  S = np.zeros((len(gamma),mc_rep,pp.n_grid))
  j = 0
  for x in X:
    k = 0
    for i in x:
      S[j,k,:] = i
      k += 1
    j += 1


  #Saving the Data
  np.save(DATA_PATH + 'CIR_' + name + '_params' + '.npy',pp)
  np.save(DATA_PATH + 'CIR_' + name + '_values' + '.npy',gamma)
  np.save(DATA_PATH + 'CIR_' + name + '.npy',S)
  print(f"Data saved to {DATA_PATH} succefully. \n")
  end_time = time.time()-start_time
  print(f"The simulation finished in {end_time} seconds.\n")